In [140]:
def analyse_exploration():
    global EXPLORATION
    input_samples = []
    output_samples = []
    games = []
    for i in range(10000):
        game = generate_random_game()
        games.append(game)
        input_samples.append(generate_input(*game))
        output_samples.append(calc_correct_output_sample(*game))        
    
    prev_exp = EXPLORATION
    for i in range(1, 2):   
        EXPLORATION = i    
        correct = []

        for s in range(100, 101, 100):
            correct.append(0)
            for g in range(len(games)): 
                set_env(*games[g])
                root = State(1, env.get_state(), env.current_player)
                _, output = mcts_game_step(root, s)

                correct_output = output_samples[g]
                correct[-1] += correct_output[np.argmax(output)] == correct_output[np.argmax(correct_output)]
        print(i, np.array(correct) / len(games))


    EXPLORATION = prev_exp

In [141]:
analyse_exploration()

1 [0.9547]


In [144]:
def eval_mcts(model, eval_cache_input, eval_cache_output):
    
    for s in range(5, 11, 2):
        correct = 0
        wrong = []
        for i in tqdm(range(len(eval_cache_input))):

            outputs = np.zeros((s, 32))
            for g in range(s):
                game = generate_game_from_obs(eval_cache_input[i])
                set_env(*game)
                root = State(1, env.get_state(), env.current_player)
                _, p = mcts_game_step(root, 1000, use_model=False)

                set_env(*game)
                for n, card in enumerate(env.players[env.current_player].hand_cards):
                    outputs[g][card.id] = p[n]

            outputs = np.mean(outputs, axis=0)
            correct_output = eval_cache_output[i]

            if correct_output[np.argmax(outputs)] == correct_output[np.argmax(correct_output)]:
                correct += 1
            else:
                wrong.append(i)
            
            #if correct_output[np.argmax(outputs)] != correct_output[np.argmax(correct_output)]:
            #   print(i, correct_output[np.argmax(outputs)], correct_output[np.argmax(correct_output)])

        print(s, correct / len(eval_cache_input), wrong)


In [ ]:
eval_mcts(model, eval_cache_input, eval_cache_output)

  0%|          | 0/1000 [00:00<?, ?it/s]

5 0.76 [3, 11, 12, 14, 19, 20, 21, 26, 27, 31, 32, 37, 58, 61, 85, 87, 92, 104, 107, 110, 113, 114, 124, 129, 135, 140, 142, 150, 152, 160, 161, 171, 176, 177, 179, 181, 189, 191, 192, 193, 196, 197, 205, 206, 218, 220, 222, 225, 226, 227, 228, 230, 232, 233, 235, 237, 243, 244, 247, 248, 250, 254, 255, 257, 266, 268, 269, 272, 284, 289, 293, 297, 305, 309, 312, 314, 318, 321, 322, 323, 324, 330, 343, 346, 347, 350, 354, 355, 358, 363, 365, 369, 372, 376, 380, 383, 386, 387, 389, 396, 397, 402, 403, 414, 421, 422, 425, 436, 437, 441, 442, 446, 447, 448, 455, 456, 460, 462, 472, 474, 478, 484, 486, 489, 495, 497, 500, 501, 503, 505, 507, 513, 515, 527, 529, 534, 539, 543, 545, 548, 553, 554, 557, 565, 568, 569, 573, 578, 579, 580, 583, 594, 603, 605, 609, 618, 621, 622, 624, 626, 630, 631, 639, 640, 642, 646, 653, 657, 658, 660, 663, 667, 672, 679, 680, 690, 691, 695, 697, 716, 720, 726, 728, 729, 734, 735, 736, 747, 752, 766, 773, 775, 776, 782, 786, 790, 793, 796, 798, 803, 806, 812, 

 93%|█████████▎| 927/1000 [10:46<00:50,  1.43it/s]

In [291]:
def analyse_samples():
    correct = 0
    wrong = []
    for i in tqdm(range(number_of_samples)):
        p = calc_correct_output_sample(*generate_game_from_obs([sample_inputs[0][i], sample_inputs[1][i]]))
        
        correct_output = np.zeros((32,))
        for n, card in enumerate(env.players[env.current_player].hand_cards):
            correct_output[card.id] = p[n]
            
        if correct_output[np.argmax(sample_outputs[0][i])] == correct_output[np.argmax(correct_output)]:
            correct += 1
        else:
            wrong.append(i)
            
    print(correct / number_of_samples, wrong)

In [292]:
analyse_samples()

100%|██████████| 7504/7504 [05:32<00:00, 22.59it/s]

0.9381663113006397 [0, 5, 22, 25, 47, 55, 60, 65, 130, 136, 137, 141, 150, 172, 179, 185, 193, 195, 227, 241, 247, 251, 262, 266, 292, 310, 347, 407, 421, 426, 438, 458, 459, 467, 510, 513, 526, 528, 552, 566, 568, 580, 618, 625, 627, 642, 666, 673, 697, 711, 753, 772, 794, 797, 798, 811, 817, 830, 850, 851, 861, 875, 891, 903, 929, 945, 946, 958, 961, 976, 984, 1053, 1059, 1109, 1134, 1139, 1148, 1150, 1177, 1186, 1189, 1190, 1203, 1210, 1212, 1214, 1257, 1263, 1265, 1279, 1285, 1293, 1295, 1299, 1305, 1321, 1322, 1331, 1342, 1390, 1394, 1418, 1421, 1448, 1461, 1488, 1510, 1512, 1519, 1532, 1554, 1557, 1560, 1563, 1564, 1586, 1590, 1594, 1612, 1613, 1624, 1631, 1632, 1654, 1656, 1719, 1738, 1743, 1773, 1795, 1797, 1811, 1820, 1823, 1828, 1829, 1843, 1854, 1859, 1884, 1892, 1906, 1926, 1930, 1959, 2033, 2058, 2064, 2112, 2113, 2132, 2144, 2170, 2174, 2178, 2199, 2201, 2202, 2209, 2220, 2223, 2239, 2256, 2260, 2261, 2295, 2306, 2314, 2324, 2325, 2344, 2366, 2378, 2439, 2450, 2476, 2615,

In [264]:
generate_game_from_obs(eval_cache_input[0])[0][1].value

<Value.SAU: 7>

In [151]:
eval_mcts(model, eval_cache_input, eval_cache_output)

100%|██████████| 1000/1000 [04:55<00:00,  3.39it/s]

3 0.743 [1, 18, 25, 28, 31, 37, 46, 48, 51, 52, 55, 57, 60, 62, 63, 67, 76, 81, 88, 89, 96, 99, 100, 102, 104, 106, 112, 114, 115, 116, 118, 122, 125, 129, 132, 133, 136, 139, 140, 142, 144, 145, 146, 151, 154, 162, 164, 167, 169, 175, 184, 194, 197, 200, 202, 209, 212, 214, 216, 219, 220, 222, 227, 232, 235, 236, 237, 240, 242, 243, 251, 255, 256, 263, 267, 275, 276, 280, 282, 284, 286, 289, 295, 300, 306, 307, 308, 311, 314, 316, 318, 322, 325, 328, 332, 338, 348, 354, 356, 359, 360, 370, 371, 377, 378, 379, 380, 383, 384, 392, 393, 394, 396, 402, 403, 408, 414, 427, 429, 433, 445, 450, 452, 455, 464, 467, 471, 475, 479, 481, 483, 485, 486, 490, 495, 496, 502, 503, 509, 516, 517, 519, 522, 525, 530, 538, 541, 543, 548, 551, 552, 559, 567, 571, 580, 593, 599, 601, 609, 617, 619, 620, 628, 630, 635, 636, 643, 651, 653, 656, 660, 664, 671, 672, 674, 676, 680, 687, 690, 697, 699, 700, 703, 707, 708, 711, 712, 713, 714, 717, 721, 726, 733, 739, 741, 742, 743, 747, 749, 758, 763, 771, 772,

In [88]:
eval_mcts(model, eval_cache_input, eval_cache_output)


21 0.86 [3, 8, 18, 24, 27, 28, 52, 54, 62, 75, 77, 87, 88, 97]
23 0.83 [3, 8, 11, 14, 24, 27, 28, 37, 54, 66, 75, 77, 86, 87, 88, 95, 97]
25 0.84 [1, 3, 4, 8, 24, 27, 28, 37, 54, 74, 75, 77, 87, 88, 95, 97]
27 0.86 [3, 8, 24, 28, 37, 40, 54, 66, 74, 75, 77, 87, 88, 97]
29 0.84 [1, 3, 8, 11, 14, 24, 27, 28, 37, 54, 74, 75, 77, 87, 88, 97]


In [143]:
eval_mcts(model, eval_cache_input, eval_cache_output)

100 0.85 [3, 16, 24, 27, 28, 62, 74, 75, 81, 86, 87, 88, 95, 97, 98]


In [106]:
val = []
for i in range(1000):
    set_env(*generate_game_from_obs(eval_cache_input[77]))
    val.append(search(env.regenerate_obs())[1])
np.mean(val, 0)

array([0.989, 0.973, 0.691])

In [166]:
show_mcts([
    card(Color.HERZ, Value.ZEHN),
    card(Color.GRUEN, Value.SAU),
    card(Color.SCHELLN, Value.SIEBEN),
    card(Color.SCHELLN, Value.NEUN),
    card(Color.EICHEL, Value.UNTER)
], None, 0, 0, [
    card(Color.EICHEL, Value.ZEHN),
    card(Color.HERZ, Value.SAU),
    card(Color.SCHELLN, Value.OBER),
    card(Color.SCHELLN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG)
])

FigureCanvasNbAgg()

array([0.09027824, 0.25778008, 0.30616403, 0.17416755, 0.1716101 ])

In [185]:
%lprun -f mcts_sample show_mcts([card(Color.HERZ, Value.ZEHN),card(Color.GRUEN, Value.SAU),card(Color.SCHELLN, Value.SIEBEN),card(Color.SCHELLN, Value.NEUN),card(Color.EICHEL, Value.UNTER)])


Timer unit: 1e-06 s

Total time: 0.438474 s
File: <ipython-input-133-7f0ddb0fbb17>
Function: mcts_sample at line 39

Line #      Hits         Time  Per Hit   % Time  Line Contents
    39                                           def mcts_sample(state, use_model=True):
    40                                               
    41       385        850.0      2.2      0.2      if state.is_leaf_node():
    42       100        106.0      1.1      0.0          if state.end_v != 0:
    43                                                       v = state.end_v
    44                                                       player = -1
    45                                                   else:
    46       100        743.0      7.4      0.2              env.set_state(state.env_state)
    47                                           
    48       100        101.0      1.0      0.0              if use_model:
    49       100     357269.0   3572.7     81.5                  p, v = model.predict_singl

In [144]:
model.predict_single(generate_input([
    card(Color.HERZ, Value.ZEHN),
    card(Color.GRUEN, Value.SAU),
    card(Color.SCHELLN, Value.NEUN),
    card(Color.EICHEL, Value.UNTER)
], None, 1, 0, [
    card(Color.HERZ, Value.SAU),
    card(Color.SCHELLN, Value.OBER),
    card(Color.SCHELLN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG)
]))

[array([0.00846176, 0.00365888, 0.00696691, 0.32488713, 0.00436447,
        0.00803394, 0.00630486, 0.00651327, 0.25214696, 0.00646617,
        0.0053455 , 0.00681768, 0.00575565, 0.00709505, 0.00533196,
        0.00393214, 0.00878035, 0.00780082, 0.00432327, 0.00539902,
        0.2030162 , 0.00293943, 0.00468038, 0.01278391, 0.01050032,
        0.00696515, 0.00696844, 0.01051198, 0.008347  , 0.3725077 ,
        0.00666502, 0.00507753], dtype=float32),
 array([0.7587035], dtype=float32)]